In [198]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [199]:
# 'apple' : input_data => apple, output_data = pple!

In [200]:
input_str = 'apple'
label_str = 'pple!'
char_set = sorted(list(set(input_str+label_str)))
print(char_set)
char_set_size = len(char_set)
print(char_set_size)

['!', 'a', 'e', 'l', 'p']
5


In [201]:
input_size = 5
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [202]:
char_to_index = dict((c,i) for i, c in enumerate(char_set))

# char_list = [c for c in char_set]
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [203]:
char_to_index.items()

dict_items([('!', 0), ('a', 1), ('e', 2), ('l', 3), ('p', 4)])

In [204]:
char_to_index.values()

dict_values([0, 1, 2, 3, 4])

In [205]:
char_to_index.keys()

dict_keys(['!', 'a', 'e', 'l', 'p'])

In [206]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value]=key

print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [207]:
x_data = [char_to_index[c] for c in input_str]      # apple
y_data = [char_to_index[c] for c in label_str]      # pple!
x_data=[x_data]
y_data=[y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [208]:
x_one_hot = [np.eye(char_set_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [209]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


In [210]:
print(X.size(), Y.size())

torch.Size([1, 5, 5]) torch.Size([1, 5])


In [211]:
# nn.RNN(input_size, hidden_size, output_size)

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn=nn.RNN(input_size, hidden_size, batch_first=True)
        # batch_first : 입력 데이터의 배치의 차원의 위치를 설정
        # (배치크기, 시퀀스 길이, 특징 수)
        # 입력의 형태가 (32,10,50) : 배치의 크기 32, 시퀀스 길이 10, 특징의 수 50
        self.fc=nn.Linear(hidden_size, output_size, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [212]:
model = Net(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [213]:
output = model(X)
print(output)
print(output.size())

tensor([[[ 0.2821,  0.1879,  0.2407,  0.0499, -0.6567],
         [ 0.2661,  0.4588, -0.1971,  0.0504, -0.3427],
         [ 0.2973,  0.3605, -0.0874, -0.0676, -0.5564],
         [ 0.2759,  0.2826, -0.1807, -0.1680, -0.3125],
         [ 0.4346,  0.3731,  0.1290,  0.0153, -0.7647]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 5, 5])


In [214]:
result = output.data.numpy().argmax(axis=2)
print(result)

[[0 1 1 1 0]]


In [215]:
for i in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = output.data.numpy().argmax(axis=2)
    str_result=''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, str_result, 'True: ', y_data)

0 loss:  1.8378946781158447 prediction:  [[0 1 1 1 0]] !aaa! True:  [[4, 4, 3, 2, 0]]
1 loss:  1.522396445274353 prediction:  [[2 4 2 2 2]] epeee True:  [[4, 4, 3, 2, 0]]
2 loss:  1.3003534078598022 prediction:  [[3 4 4 2 2]] lppee True:  [[4, 4, 3, 2, 0]]
3 loss:  1.095206618309021 prediction:  [[4 4 4 2 0]] pppe! True:  [[4, 4, 3, 2, 0]]
4 loss:  0.8944276571273804 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
5 loss:  0.6862616539001465 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
6 loss:  0.4943733215332031 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
7 loss:  0.3355202078819275 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
8 loss:  0.21703490614891052 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
9 loss:  0.14156194031238556 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
10 loss:  0.09676893055438995 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
11 loss:  0.06847016513347626 prediction:  [[4 4 3 2

126 loss:  0.0003034995752386749 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
127 loss:  0.00030190288089215755 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
128 loss:  0.0003004252794198692 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
129 loss:  0.0002989000640809536 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
130 loss:  0.00029744632774963975 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
131 loss:  0.0002959448902402073 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
132 loss:  0.00029446728876791894 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
133 loss:  0.0002930135524366051 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
134 loss:  0.00029155975789763033 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
135 loss:  0.0002901536354329437 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
136 loss:  0.0002886998699977994 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2,

In [216]:
sentence = """if you want to build a ship, don't drum up people together to
collect wood and don't assign them taskss and work, but rather
teach them to long for the endless immensity of the sea."""

In [217]:
char_set1 = list(set(sentence))
print(char_set1)

['y', '.', 'e', 'o', 'h', 'p', 'b', 't', 'd', '\n', 'k', 's', 'g', 'l', 'a', 'u', 'f', 'r', ',', 'm', "'", 'i', 'n', 'w', 'c', ' ']


In [218]:
char_dic = {c:i for i, c in enumerate(char_set1)}
char_dic

{'y': 0,
 '.': 1,
 'e': 2,
 'o': 3,
 'h': 4,
 'p': 5,
 'b': 6,
 't': 7,
 'd': 8,
 '\n': 9,
 'k': 10,
 's': 11,
 'g': 12,
 'l': 13,
 'a': 14,
 'u': 15,
 'f': 16,
 'r': 17,
 ',': 18,
 'm': 19,
 "'": 20,
 'i': 21,
 'n': 22,
 'w': 23,
 'c': 24,
 ' ': 25}

In [219]:
char_dic_size = len(char_dic)
print(char_dic_size)

26


In [220]:
hidden_size = char_dic_size
sequence_length = 10        # 훈련데이터 batch size
learning_rate = 0.1


In [221]:
x_data = []
y_data = []

for i in range(0, len(sentence)-sequence_length):
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1 : i+sequence_length+1]
    print(i, x_str,':', y_str)
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan : f you want
1 f you want :  you want 
2  you want  : you want t
3 you want t : ou want to
4 ou want to : u want to 
5 u want to  :  want to b
6  want to b : want to bu
7 want to bu : ant to bui
8 ant to bui : nt to buil
9 nt to buil : t to build
10 t to build :  to build 
11  to build  : to build a
12 to build a : o build a 
13 o build a  :  build a s
14  build a s : build a sh
15 build a sh : uild a shi
16 uild a shi : ild a ship
17 ild a ship : ld a ship,
18 ld a ship, : d a ship, 
19 d a ship,  :  a ship, d
20  a ship, d : a ship, do
21 a ship, do :  ship, don
22  ship, don : ship, don'
23 ship, don' : hip, don't
24 hip, don't : ip, don't 
25 ip, don't  : p, don't d
26 p, don't d : , don't dr
27 , don't dr :  don't dru
28  don't dru : don't drum
29 don't drum : on't drum 
30 on't drum  : n't drum u
31 n't drum u : 't drum up
32 't drum up : t drum up 
33 t drum up  :  drum up p
34  drum up p : drum up pe
35 drum up pe : rum up peo
36 rum up peo : um up peop
37 um up pe

In [222]:
print(x_data)
print(y_data)

[[21, 16, 25, 0, 3, 15, 25, 23, 14, 22], [16, 25, 0, 3, 15, 25, 23, 14, 22, 7], [25, 0, 3, 15, 25, 23, 14, 22, 7, 25], [0, 3, 15, 25, 23, 14, 22, 7, 25, 7], [3, 15, 25, 23, 14, 22, 7, 25, 7, 3], [15, 25, 23, 14, 22, 7, 25, 7, 3, 25], [25, 23, 14, 22, 7, 25, 7, 3, 25, 6], [23, 14, 22, 7, 25, 7, 3, 25, 6, 15], [14, 22, 7, 25, 7, 3, 25, 6, 15, 21], [22, 7, 25, 7, 3, 25, 6, 15, 21, 13], [7, 25, 7, 3, 25, 6, 15, 21, 13, 8], [25, 7, 3, 25, 6, 15, 21, 13, 8, 25], [7, 3, 25, 6, 15, 21, 13, 8, 25, 14], [3, 25, 6, 15, 21, 13, 8, 25, 14, 25], [25, 6, 15, 21, 13, 8, 25, 14, 25, 11], [6, 15, 21, 13, 8, 25, 14, 25, 11, 4], [15, 21, 13, 8, 25, 14, 25, 11, 4, 21], [21, 13, 8, 25, 14, 25, 11, 4, 21, 5], [13, 8, 25, 14, 25, 11, 4, 21, 5, 18], [8, 25, 14, 25, 11, 4, 21, 5, 18, 25], [25, 14, 25, 11, 4, 21, 5, 18, 25, 8], [14, 25, 11, 4, 21, 5, 18, 25, 8, 3], [25, 11, 4, 21, 5, 18, 25, 8, 3, 22], [11, 4, 21, 5, 18, 25, 8, 3, 22, 20], [4, 21, 5, 18, 25, 8, 3, 22, 20, 7], [21, 5, 18, 25, 8, 3, 22, 20, 7, 25]

In [223]:
x_one_hot = [np.eye(char_dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print(X)
print(Y)
print(X.size(), Y.size())

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        ...,

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [224]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn=nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc=nn.Linear(hidden_dim, hidden_dim, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [225]:
model = Net(char_dic_size, hidden_size, 2)      # 층을 2개 쌓겠다는 뜻
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [226]:
outputs = model(X)
print(outputs)

tensor([[[ 0.1358, -0.0591,  0.1026,  ..., -0.0444, -0.2691,  0.1583],
         [ 0.0294,  0.0673,  0.1345,  ..., -0.0287, -0.1551,  0.0208],
         [ 0.1630,  0.0115,  0.0575,  ..., -0.0972, -0.1971,  0.0781],
         ...,
         [ 0.2018,  0.0492,  0.1676,  ..., -0.0973, -0.1571,  0.1110],
         [ 0.2054, -0.1315,  0.1182,  ...,  0.0365, -0.3271,  0.1159],
         [ 0.1664, -0.0040,  0.1825,  ...,  0.0551, -0.1835,  0.0800]],

        [[-0.0483, -0.0109,  0.1520,  ..., -0.0540, -0.1636,  0.1529],
         [ 0.1575,  0.0519,  0.0676,  ..., -0.1237, -0.1599,  0.0742],
         [ 0.1723,  0.0164,  0.2101,  ..., -0.0200, -0.2012,  0.0836],
         ...,
         [ 0.2076, -0.1287,  0.1205,  ...,  0.0347, -0.3250,  0.1152],
         [ 0.1664, -0.0040,  0.1826,  ...,  0.0554, -0.1835,  0.0784],
         [ 0.2009, -0.0713,  0.2334,  ..., -0.0220, -0.2672,  0.0656]],

        [[ 0.0615, -0.0623,  0.0553,  ..., -0.0856, -0.2312,  0.1577],
         [ 0.1521,  0.0351,  0.1192,  ..., -0

In [227]:
print(outputs.size())

torch.Size([171, 10, 26])


In [228]:
# result = outputs.view(-1, char_dic_size)
# print(result.size())

In [229]:
# .join의 역할
a = ['p','y','t','h','o','n']
a1 = ''.join(a)
a1

'python'

In [231]:
for i in range(200):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.view(-1, char_dic_size), Y.view(-1))

    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""

    for j, result in enumerate(results):
        if j==0 :
            predict_str+=''.join([char_set1[t] for t in result])
        else:
            predict_str+=char_set1[result[-1]]
        print(predict_str)

g sormuort
g sormuort 
g sormuort t
g sormuort th
g sormuort th 
g sormuort th b
g sormuort th bo
g sormuort th boi
g sormuort th boil
g sormuort th boild
g sormuort th boild 
g sormuort th boild a
g sormuort th boild an
g sormuort th boild ans
g sormuort th boild ansh
g sormuort th boild ansh'
g sormuort th boild ansh'g
g sormuort th boild ansh'g,
g sormuort th boild ansh'g, 
g sormuort th boild ansh'g, l
g sormuort th boild ansh'g, lo
g sormuort th boild ansh'g, lon
g sormuort th boild ansh'g, lon'
g sormuort th boild ansh'g, lon't
g sormuort th boild ansh'g, lon't 
g sormuort th boild ansh'g, lon't a
g sormuort th boild ansh'g, lon't ao
g sormuort th boild ansh'g, lon't aou
g sormuort th boild ansh'g, lon't aoum
g sormuort th boild ansh'g, lon't aoum 
g sormuort th boild ansh'g, lon't aoum p
g sormuort th boild ansh'g, lon't aoum pp
g sormuort th boild ansh'g, lon't aoum pp 
g sormuort th boild ansh'g, lon't aoum pp p
g sormuort th boild ansh'g, lon't aoum pp pe
g sormuort th boild 